In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer      #experimental api
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import xgboost as xgb

In [43]:
train_data=pd.read_csv('datasets/train.csv')
print(train_data.info())
data=np.array(train_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [44]:
categorical_data=train_data.select_dtypes(include=['object'])
numerical_data=train_data.select_dtypes(exclude=['object'])
# print(categorical_data.info())
# print(numerical_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   Gender              1200000 non-null  object
 1   Marital Status      1181471 non-null  object
 2   Education Level     1200000 non-null  object
 3   Occupation          841925 non-null   object
 4   Location            1200000 non-null  object
 5   Policy Type         1200000 non-null  object
 6   Policy Start Date   1200000 non-null  object
 7   Customer Feedback   1122176 non-null  object
 8   Smoking Status      1200000 non-null  object
 9   Exercise Frequency  1200000 non-null  object
 10  Property Type       1200000 non-null  object
dtypes: object(11)
memory usage: 100.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype  
---  ------            

In [45]:
cat_col=[]
num_col=[]
for i in train_data.columns:
    if train_data[i].dtype=='object':
        cat_col.append(i)
    else:
        num_col.append(i)

# print('categorical col: '+str(cat_col))
# print('numerical col: '+str(num_col))


categorical col: ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Policy Start Date', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']
numerical col: ['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Premium Amount']


In [46]:
train_data["Policy Start Date"]=pd.to_datetime(train_data["Policy Start Date"])

train_data["year"]=train_data["Policy Start Date"].dt.year
train_data["month"]=train_data["Policy Start Date"].dt.month
train_data["day"]=train_data["Policy Start Date"].dt.day

train_data.drop("Policy Start Date",axis=1,inplace=True)

In [47]:
num_col.append("year")
num_col.append("month")
num_col.append("day")
cat_col.remove("Policy Start Date")

In [48]:
num_inputer=SimpleImputer(strategy='mean')
cat_inputer=SimpleImputer(strategy='most_frequent')

train_data[num_col]=num_inputer.fit_transform(train_data[num_col])
train_data[cat_col]=cat_inputer.fit_transform(train_data[cat_col])

# print(train_data.isnull().sum())


id                      0
Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Insurance Duration      0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
year                    0
month                   0
day                     0
dtype: int64


In [49]:
encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
# for i in cat_col:
#   if(i in train_data.columns):
#       print(i)
for i in cat_col:
  # print(i)
  encoded_value=encoder.fit_transform(train_data[[i]])
  encoded_value=pd.DataFrame(encoded_value,columns=encoder.get_feature_names_out([i]))
  train_data.drop(i,axis=1,inplace=True)
  train_data=pd.concat([train_data,encoded_value],axis=1)

In [50]:
y=np.log1p(train_data["Premium Amount"])
num=train_data.columns
scaler=StandardScaler()
train_data[num]=scaler.fit_transform(train_data[num])

In [37]:
print((train_data["Premium Amount"] < 0).sum())  

734809


In [14]:
correration_matrix=train_data.corr()
# print(correration_matrix)

                                   id       Age  Annual Income  \
id                           1.000000 -0.000134      -0.000870   
Age                         -0.000134  1.000000      -0.000009   
Annual Income               -0.000870 -0.000009       1.000000   
Number of Dependents         0.000717  0.001395       0.001993   
Health Score                 0.001370  0.000847       0.023995   
Previous Claims             -0.000135  0.001548       0.035647   
Vehicle Age                 -0.001461 -0.002436      -0.000490   
Credit Score                 0.000915  0.002652      -0.186491   
Insurance Duration          -0.000350 -0.000063       0.000372   
Premium Amount              -0.000292 -0.002411      -0.012091   
year                        -0.000912 -0.003152      -0.009758   
month                        0.000621  0.001195       0.008857   
day                         -0.000692 -0.000595       0.000334   
Gender_Female                0.001454 -0.000215       0.000934   
Gender_Mal

In [15]:
# print(correration_matrix["Premium Amount"])
train_data.drop("id",axis=1,inplace=True)

id                            -0.000292
Age                           -0.002411
Annual Income                 -0.012091
Number of Dependents          -0.000928
Health Score                   0.014326
Previous Claims                0.039364
Vehicle Age                    0.000391
Credit Score                  -0.024438
Insurance Duration            -0.000028
Premium Amount                 1.000000
year                          -0.011084
month                          0.006702
day                            0.000291
Gender_Female                 -0.000161
Gender_Male                    0.000161
Marital Status_Divorced       -0.001545
Marital Status_Married        -0.002184
Marital Status_Single          0.003682
Education Level_Bachelor's     0.000103
Education Level_High School    0.001462
Education Level_Master's      -0.000290
Education Level_PhD           -0.001252
Occupation_Employed           -0.003727
Occupation_Self-Employed       0.002608
Occupation_Unemployed          0.001787


In [ ]:
print(train_data["Premium Amount"].isnull().sum())
print((train_data["Premium Amount"] < 0).sum())  

0
734809


In [51]:
X=train_data.drop("Premium Amount",axis=1)

Now implement a regressor

In [52]:
best_params = {
    'colsample_bytree': 0.9,
    'learning_rate': 0.01,
    'max_depth': 7,
    'n_estimators': 500,
    'subsample': 0.8
}

model=xgb.XGBRegressor(**best_params)

In [53]:
model.fit(X,y)
predictions=model.predict(X)

accuracy=model.score(X,y)
print(accuracy)

0.08380092957600915


In [ ]:

prediction=np.expm1(predictions)
result=pd.DataFrame({"id":train_data["id"],"Premium Amount":prediction})
result.to_csv("results/prediction.csv",index=False)